In [34]:
from collections import Counter, defaultdict, deque
from functools import partial
from itertools import chain, cycle, product

from joblib import Parallel, delayed

import numpy as np

In [35]:
# consider lanes with largest index first
def lane_index_priority(lane_tuple, product=None, boxes=None):

    if product is None or boxes is None:
        raise RuntimeError("No product or boxes set")

    return (lane_tuple[0], -boxes[lane_tuple[0]])


# consider lane with largest sum number of products first
def sum_of_products_priority(lane_tuple, product=None, boxes=None):

    if product is None or boxes is None:
        raise RuntimeError("No product or boxes set")

    return (sum(lane_tuple[1].values()), -boxes[lane_tuple[0]])


# consider lane with largest product count first, ex: largest number of As
def product_count_priority(lane_tuple, product=None, boxes=None):

    if product is None or boxes is None:
        raise RuntimeError("No product or boxes set")

    return (lane_tuple[1][product], -boxes[lane_tuple[0]])


# priority based on randomly generated numbers
def random_priority(lane_tuple, product=None, boxes=None):

    if product is None or boxes is None:
        raise RuntimeError("No product or boxes set")

    return (np.random.rand(), -boxes[lane_tuple[0]])

In [36]:
def generate_sequential_batches(batch_size, recipe, sample_size):
    products = list(recipe.keys())
    totals = list(recipe.values())
    num_of_batches = np.ceil(sample_size / batch_size).astype(int)

    counts = dict(sorted(zip(products, totals), key=lambda x: x[1], reverse=True))

    pattern = []
    while 0 < sum(counts.values()):
        for product in counts.keys():
            if 0 < counts[product]:
                pattern.append(product)
                counts[product] -= 1

    out = []
    for index, product in zip(range(num_of_batches), cycle(pattern)):
        out.append(product)

    return out


def generate_random_products(recipe, sample_size):
    products = list(recipe.keys())
    return np.random.choice(
        products,
        size=sample_size,
        p=[t / sum(recipe.values()) for t in recipe.values()],
    ).tolist()

In [62]:
recipes = [
    {"A": 8, "B": 4, "C": 4, "D": 8, "E": 10, "F": 16},
    {"A": 10, "B": 10, "C": 6, "D": 4},
    {"A": 3, "B": 3, "C": 4, "D": 3, "E": 3, "F": 2},
]
batch_sizes = [25, 50, 100]
sample_size = 30000
lane_lock = 25
priority = product_count_priority
products_per_min = 250

In [63]:
def create_infeed_types():
    sequences = {}

    for i, batch_size in product(range(len(recipes)), batch_sizes):
        sequences[(i, "sequential_batches", batch_size)] = generate_sequential_batches(
            batch_size, recipes[i], sample_size
        )

    for i, recipe in enumerate(recipes):
        sequences[(i, "random_products", None)] = generate_random_products(
            recipe, sample_size
        )

    return sequences


sequences = create_infeed_types()

In [64]:
from numba import jit, njit
import numpy as np

In [65]:
num_of_products = 6

num_of_lanes = 5

infeed = np.random.randint(num_of_products, size=sample_size)

recipe = np.array([8, 4, 4, 8, 10, 16])

In [131]:
@njit(cache=True)
def np_all_axis1(x):
    # Numba compatible version of np.all(x, axis=1)
    out = np.ones(x.shape[0], dtype=np.bool8)
    for i in range(x.shape[1]):
        out = np.logical_and(out, x[:, i])
    return out


@njit(cache=True)
def np_lexsort(x, y):
    # Numba compatible version of np.lexsort((y,x))
    values = sorted(zip(x, y, range(len(x))))
    return np.array([v[-1] for v in values])


@njit
def simulate_fast(num_of_lanes, num_of_products, recipe, infeed):
    # rows are lanes
    # columns are products
    # n, m := num_of_lanes, num_of_products
    
    # n x m := store the state of each lane with product counts
    states = np.zeros((num_of_lanes, num_of_products))
    
    # n x 1 := store number of boxes produced per lane
    production = np.zeros(num_of_lanes)
    
    # n x 1 := store lock value of a lane
    locks = np.zeros(num_of_lanes)
    
    # count the amount of products that don't fit
    recirculation = 0

    # TODO := consider iterating over the infeed (no index)
    for i in range(infeed.size):
    
        product = infeed[i]

        # n x 1 := lanes that are unlocked and available to recieve
        available = locks == 0

        # n x 1 := lanes that have enough space to fit the product
        fits = states[:,product] < recipe[product]

        # n x 1 := lane indeces ordered by priority
        # priority := highest product count, then lowest production
        # order = np.lexsort((production, -states[:,product]))
        order = np_lexsort(-states[:,product], production)
        
        # n x 1 := ordered boolean mask of available and fit lanes
        # NOTE - can only be used if lanes are ordered
        mask = (available & fits)[order]

        if 0 < order[mask].size:
            lane = order[mask][0]
            states[lane,product] += 1
            consumed = True
        else:
            recirculation += 1            

        # decrease lock value for unavailable lanes
        locks[~available] -= 1

        # n x 1 := lanes that are ready for processing
        # full = np.all(states == recipe, axis=1)
        full = np_all_axis1(states == recipe)

        # reset state, increase production value and set lane lock
        states[full,:] = 0
        production[full] += 1
        locks[full] = 25
        
    return recirculation

In [132]:
%timeit simulate_fast(num_of_lanes, num_of_products, recipe, infeed)

38.8 ms ± 506 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [133]:
%timeit simulate_fast(num_of_lanes, num_of_products, recipe, infeed)

38.6 ms ± 95.6 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [100]:
def simulate_numpy(num_of_lanes, num_of_products, recipe, infeed):
    # rows are lanes
    # columns are products
    # n, m := num_of_lanes, num_of_products
    
    # n x m := store the state of each lane with product counts
    states = np.zeros((num_of_lanes, num_of_products))
    
    # n x 1 := store number of boxes produced per lane
    production = np.zeros(num_of_lanes)
    
    # n x 1 := store lock value of a lane
    locks = np.zeros(num_of_lanes)
    
    # count the amount of products that don't fit
    recirculation = 0

    # TODO := consider iterating over the infeed (no index)
    for i in range(infeed.size):

        product = infeed[i]

        # n x 1 := lanes that are unlocked and available to recieve
        available = locks == 0

        # n x 1 := lanes that have enough space to fit the product
        fits = states[:,product] < recipe[product]

        # n x 1 := lane indeces ordered by product count priority
        # last key is 1st order, 2nd-to-last key is 2nd order, and so on
        # highest product count, then lowest production
        order = np.lexsort((production, -states[:,product]))
        
        # n x 1 := ordered boolean mask of available and fit lanes
        # NOTE - can only be used if lanes are ordered
        mask = (available & fits)[order]

        if 0 < order[mask].size:
            lane = order[mask][0]
            states[lane,product] += 1
        else:
            recirculation += 1            

        # decrease lock value for unavailable lanes
        locks[~available] -= 1

        # n x 1 := lanes that are ready for processing
        full = np.all(states == recipe.reshape(1, -1), axis=1)

        # reset state, increase production value and set lane lock
        states[full,:] = 0
        production[full] += 1
        locks[full] = 25
        
    return recirculation

In [101]:
%timeit simulate_numpy(num_of_lanes, num_of_products, recipe, infeed)

454 ms ± 2.36 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [102]:
%timeit simulate_numpy(num_of_lanes, num_of_products, recipe, infeed)

465 ms ± 5.56 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [134]:
121 / 39

3.1025641025641026

In [72]:
def simulate(num_of_lanes, infeed_type, recipe_index, batch_size=None):

    # split recipes into products and totals
    recipe = recipes[recipe_index]
    products = list(recipe.keys())
    totals = list(recipe.values())

    # Initialize data containers
    batch_order = []
    lanes = {i: {p: 0 for p in products + ["lock"]} for i in range(num_of_lanes)}
    boxes_per_lane = {i: 0 for i in range(num_of_lanes)}
    boxes_per_lane_per_frame = {i: [] for i in range(num_of_lanes)}
    recirculation = []

    # Initialize product queue will get updated based on infeed system
    product_queue = deque()

    # init product queue base on infeed type
    if infeed_type == "sequential_batches":

        if batch_size is None:
            raise RuntimeError(f"batch_size must be specified")

        batches_queue = deque(sequences[(recipe_index, infeed_type, batch_size)])
        next_batch = batches_queue.popleft()
        batch_order.append(next_batch)

        product_queue.extend([next_batch] * batch_size)

    elif infeed_type == "random_products":

        product_queue.extend(sequences[(recipe_index, infeed_type, None)])

    else:
        raise RuntimeError(f"infeed_type {infeed_type} if unknown!")

    product_counter = 0
    while product_counter < sample_size:

        product = product_queue.popleft()
        consumed = False
        for i, lane in sorted(
            lanes.items(),
            key=partial(priority, product=product, boxes=boxes_per_lane),
            reverse=True,
        ):

            # Lane is available to accept products
            lane_is_available = lane["lock"] == 0

            # Lane has space to accept product
            product_fits = lane[product] < recipe[product]

            # Complete cases for 3 binary variables
            if not consumed and lane_is_available and product_fits:
                lane[product] += 1
                consumed = True

            elif not consumed and lane_is_available and not product_fits:
                continue

            elif not consumed and not lane_is_available and product_fits:
                lane["lock"] -= 1

            elif not consumed and not lane_is_available and not product_fits:
                lane["lock"] -= 1

            elif consumed and lane_is_available and product_fits:
                continue

            elif consumed and lane_is_available and not product_fits:
                continue

            elif consumed and not lane_is_available and product_fits:
                lane["lock"] -= 1

            elif consumed and not lane_is_available and not product_fits:
                lane["lock"] -= 1

        # recirculation un-consumed products
        if not consumed:
            recirculation.append(product)

        # Lane is ready to be boxed
        for j in lanes:
            boxes_per_lane_per_frame[j].append(0)
            if all(lanes[j][p] == recipe[p] for p in products):
                lanes[j] = {p: 0 for p in products + ["lock"]}
                lanes[j]["lock"] = lane_lock
                boxes_per_lane[j] += 1
                boxes_per_lane_per_frame[j][product_counter] = 1

        # continue onto the next product
        product_counter += 1

        # check if the queue is empty
        if (
            len(product_queue) == 0
            and infeed_type == "sequential_batches"
            and product_counter < sample_size
        ):
            next_batch = batches_queue.popleft()
            batch_order.append(next_batch)
            product_queue.extend([next_batch] * batch_size)

    return {
        "recirculation": len(recirculation),
        "sample_size": sample_size,
        "box_products": sum(recipe.values()) * sum(boxes_per_lane.values()),
        "unbox_products": sample_size
        - len(recirculation)
        - sum(recipe.values()) * sum(boxes_per_lane.values()),
        "boxes_per_lane": boxes_per_lane,
        "boxes": sum(boxes_per_lane.values()),
        "boxes_per_lane_per_frame": boxes_per_lane_per_frame,
        "batch_order": batch_order,
    }

In [73]:
%timeit simulate(5, "random_products", 0)

121 ms ± 599 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
out = simulate(5, "random_products", 0)
out.pop("boxes_per_lane_per_frame")
out

In [ ]:
out = simulate(5, "sequential_batches", 0, batch_size=50)
out.pop("boxes_per_lane_per_frame")
batch_counts = Counter(out.pop("batch_order"))
print({k: round(value / min(batch_counts.values()), 2) for k, value in sorted(batch_counts.items())})
print({k: round(value / min(recipes[0].values()), 2) for k, value in sorted(recipes[0].items())})
out

In [ ]:
def get_the_ys(xx, infeed_type, recipe_index, batch_size=None):
    data = Parallel(n_jobs=4)(
        delayed(simulate)(x, infeed_type, recipe_index, batch_size=batch_size)
        for x in xx
    )
    return [d["box_products"] / d["sample_size"] for d in data]

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

fig, axs = plt.subplots(
    len(recipes), figsize=(16, 9 * len(recipes)), constrained_layout=True
)

fig.suptitle("Lane Production Analysis")

color_map = defaultdict(lambda: "k")
for batch_size, color in zip(batch_sizes, mcolors.BASE_COLORS.keys()):
    color_map[batch_size] = color

line_type_map = defaultdict(lambda: "-")

xx = list(range(1, 17))

for (recipe_index, infeed_type, batch_size) in sequences.keys():
    color = color_map[batch_size]
    line_type = line_type_map[infeed_type]

    if infeed_type in ["random_products", "sequential_batches"]:
        yy = get_the_ys(xx, infeed_type, recipe_index, batch_size)
        axs[recipe_index].plot(
            xx,
            yy,
            f"{line_type}{color}",
            marker="*",
            label=f"{infeed_type} {batch_size} {priority.__name__}",
        )

for recipe_index, recipe in enumerate(recipes):
    zz = np.ones(len(xx)) * 0.95
    axs[recipe_index].plot(xx, zz, f":k", label=f"Target consumption")
    axs[recipe_index].set_xticks(xx)
    axs[recipe_index].legend()
    axs[recipe_index].set_title(f"Simulation recipe: {recipe}")
    axs[recipe_index].set_xlabel("Num of lanes")
    axs[recipe_index].set_ylabel("Consume proportion")

plt.show()

In [ ]:
batch_size = 50
num_of_lanes = 5

In [ ]:
def create_subplot(recipe_index, infeed_type, batch_size=None):
    data = simulate(num_of_lanes, infeed_type, recipe_index, batch_size=batch_size)

    for lane, production in data["boxes_per_lane_per_frame"].items():
        arrays = np.array_split(
            production, range(products_per_min, len(production), products_per_min)
        )
        y = [a.sum() for a in arrays]
        axs[recipe_index].plot([0] + y, label=f"Lane {lane}")

    # combine all lanes
    x = np.array(list(data["boxes_per_lane_per_frame"].values())).sum(axis=0)
    arrays = np.array_split(x, range(products_per_min, len(x), products_per_min))
    y = [a.sum() for a in arrays]
    axs[recipe_index].plot([0] + y, label=f"All lanes")

    # optimality lines
    recipe = recipes[recipe_index]
    optimal_boxes_per_min = products_per_min / sum(recipe.values())
    optimal_boxes_per_min_per_lane = optimal_boxes_per_min / num_of_lanes
    axs[recipe_index].plot(
        [optimal_boxes_per_min] * (len(y) + 1), f":k", label=f"optimal_boxes_per_min"
    )
    axs[recipe_index].plot(
        [optimal_boxes_per_min_per_lane] * (len(y) + 1),
        f"-.k",
        label=f"optimal_boxes_per_min",
    )
    axs[recipe_index].legend()
    axs[recipe_index].set_title(f"Simulation recipe: {recipe}")
    axs[recipe_index].set_xlabel("Minutes")
    axs[recipe_index].set_ylabel("Boxes produced")

In [ ]:
fig, axs = plt.subplots(
    len(recipes), figsize=(16, 9 * len(recipes)), constrained_layout=True
)

fig.suptitle("Boxes Per Min Analysis with Randomized Infeed")

for recipe_index, recipe in enumerate(recipes):
    create_subplot(recipe_index, "random_products")

plt.show()

In [ ]:
fig, axs = plt.subplots(
    len(recipes), figsize=(16, 9 * len(recipes)), constrained_layout=True
)

fig.suptitle("Boxes Per Min Analysis with Sequential Infeed")

for recipe_index, recipe in enumerate(recipes):
    create_subplot(recipe_index, "sequential_batches", batch_size=batch_size)

plt.show()

In [ ]:
def avg_consumption_rate(x_num_of_lanes, y_lane_lock, infeed_type, batch_size=None):
    x, y = np.meshgrid(x_num_of_lanes, y_lane_lock)

    I, J = y.shape

    data = []
    
    for recipe_index in range(len(recipes)):
        z_arrays = []

        for i in range(I):
            global lane_lock
            lane_lock = y[i, 0]
            z_arrays.append(get_the_ys(x[i, :], infeed_type, recipe_index, batch_size=batch_size))

        data.append(np.array(z_arrays))

    tensor = np.array(data)
    
    return tensor.mean(axis=0)

In [ ]:
plt.figure(figsize=(16, 9))

x_num_of_lanes = list(range(1, 11))
# x_num_of_lanes = list(range(1, 6))
y_lane_lock = list(range(20, 100, 5)) + list(range(100, 400, 50))
# y_lane_lock = list(range(20, 100, 20))
z_avg_consumption_rate = avg_consumption_rate(x_num_of_lanes, y_lane_lock, "random_products") 

plt.contourf(x_num_of_lanes, y_lane_lock, z_avg_consumption_rate, vmin=0, vmax=1)
plt.colorbar(label="Avg. Consumption Rate")

plt.title(f"Random Infeed: avg. consumption rate vs num of lanes and lane lock")
plt.ylabel("Lane lock (frames)")
plt.xlabel("Number of Lanes")

plt.show()

In [ ]:
plt.figure(figsize=(16, 9))

x_num_of_lanes = list(range(1, 11))
# x_num_of_lanes = list(range(1, 6))
y_lane_lock = list(range(20, 100, 5)) + list(range(100, 400, 50))
# y_lane_lock = list(range(20, 100, 20))
z_avg_consumption_rate = avg_consumption_rate(x_num_of_lanes, y_lane_lock, "sequential_batches", 50) 

plt.contourf(x_num_of_lanes, y_lane_lock, z_avg_consumption_rate, vmin=0, vmax=1)
plt.colorbar(label="Avg. Consumption Rate")

plt.title(f"Sequential Infeed: avg. consumption rate vs num of lanes and lane lock")
plt.ylabel("Lane lock (frames)")
plt.xlabel("Number of Lanes")

plt.show()

In [ ]:
# reset global variable
lane_lock = 25

In [ ]:
import string


def generate_random_recipes(
    num_of_recipes, min_num_of_products, max_num_of_products, lower, upper
):
    recipes = []
    for r in range(num_of_recipes):
        recipe = {}

        for product in string.ascii_uppercase[
            : np.random.randint(min_num_of_products, high=max_num_of_products)
        ]:
            recipe[product] = np.random.randint(lower, high=upper)

        recipes.append(recipe)

    return recipes

In [ ]:
recipes = generate_random_recipes(60, 3, 6, 2, 16)
recipes

In [ ]:
sequences = create_infeed_types()

In [ ]:
plt.figure(figsize=(16, 9))

xx = list(range(1, 11))
arrays = []
for recipe_index, recipe in enumerate(recipes):
    yy = get_the_ys(xx, "random_products", recipe_index)
    arrays.append(yy)
    plt.plot(xx, yy, marker="*", label=f"Recipe: {recipe}")

target_consumption = 0.95
zz = np.ones(len(xx)) * target_consumption
plt.plot(xx, zz, f":k", label=f"Target consumption")

plt.xticks(xx)
plt.legend(bbox_to_anchor=(1.05, 1.0), loc="upper left")

plt.title(f"Random Infeed System with Target consumption {target_consumption}")
plt.xlabel("Num of lanes")
plt.ylabel("Consume proportion")

plt.show()

In [ ]:
plt.figure(figsize=(16, 9))

plt.boxplot(np.array(arrays))

zz = np.ones(len(xx)) * target_consumption
plt.plot(xx, zz, ":k")

plt.title(f"Random Infeed System with Target consumption {target_consumption}")
plt.xlabel("Num of lanes")
plt.ylabel("Consume proportion")

plt.show()